# Getting Wikipedia Data

In [ ]:
!pip install wikipedia

In [ ]:
import wikipedia as wiki

In [ ]:
search_string = 'Social Network Analysis'

page = wiki.page(search_string)

content = page.content

content[0:680]

In [ ]:
links = page.links

links[0:10]

In [ ]:
# use this if get error on next cell

import ssl
ssl._create_default_https_context = ssl._create_unverified_context

In [ ]:
import pandas as pd

url = 'https://en.wikipedia.org/wiki/Social_network_analysis'

data = pd.read_html(url)

type(data)

In [ ]:
len(data)

In [ ]:
data

In [ ]:
data[0]

In [ ]:
data[1]

In [ ]:
url = 'https://en.wikipedia.org/wiki/Crime_in_Oregon'

data = pd.read_html(url)

df = data[1]

df.tail()

In [ ]:
data[1]

In [ ]:
type(data)

In [ ]:
type(df)

# Project Gutenberg

In [ ]:
import nltk

nltk.download('gutenberg')

In [ ]:
nltk.__version__

In [ ]:
from nltk.corpus import gutenberg


gutenberg.fileids()

In [ ]:
file = 'blake-poems.txt'

data = gutenberg.raw(file)

data[0:600]

In [ ]:
# WARNING: do not crawl Gutenberg aggressively; load one book, don't download all, else IP may get blocked

import requests

url = 'https://www.gutenberg.org/files/5200/5200-0.txt'

data = requests.get(url).text

data

In [ ]:
url = 'http://english.ryukyushimpo.jp/'

data = requests.get(url).text

data

In [ ]:
from bs4 import BeautifulSoup

soup = BeautifulSoup(data, 'html.parser')

links = soup.find_all('a', href=True)

links

In [ ]:
len(links)

In [ ]:
url = 'http://english.ryukyushimpo.jp/2021/09/03/34020/'

data = requests.get(url).text

soup = BeautifulSoup(data, 'html.parser')

soup.get_text()

In [ ]:
text = soup.get_text()

text[0:500]

In [ ]:
text = text.replace('\n', ' ').replace('\r', ' ').replace('\t', ' ').replace('\xa0', ' ')

text

In [ ]:
cutoff = text.index('Go to Japanese')

cutoff

In [ ]:
text = text[0:cutoff]

In [ ]:
cutoff = text.rindex('Hirokazu Ueyonabaru')

text = text[cutoff:]

In [ ]:
text

# NLTK Part-of-Speech Tagging (Pos-Tagging)

In [ ]:
url = 'https://www.gutenberg.org/files/5200/5200-0.txt'

text = requests.get(url).text

In [ ]:
cutoff = text.index('One morning')

text = text[cutoff:]

In [ ]:
cutoff = text.rindex('*** END OF THE PROJECT GUTENBERG EBOOK METAMORPHOSIS ***')

text = text[:cutoff]

In [ ]:
text[-500:]

In [ ]:
text = text.replace('\r', ' ').replace('\n', ' ')

text[0:505]

In [ ]:
text = text.replace('â\x80\x99', '\'').replace('â\x80\x9c', '"').replace('â\x80\x9d', '""').replace('â\x80\x94', ' ')

text[0:502]

In [ ]:
def get_data():

    url = 'https://www.gutenberg.org/files/5200/5200-0.txt'
    text = requests.get(url).text

    # strip header junk
    cutoff = text.index('One morning')
    text = text[cutoff:]

    # strip footer junk
    cutoff = text.rindex('*** END OF THE PROJECT GUTENBERG EBOOK METAMORPHOSIS ***')
    text = text[:cutoff]

    # pre-processing to clean the text
    text = text.replace('\r', ' ').replace('\n', ' ')
    text = text.replace('â\x80\x99', '\'').replace('â\x80\x9c', '"')\
        .replace('â\x80\x9d', '""').replace('â\x80\x94', ' ')

    return text

In [ ]:
text = get_data()

text[0:502]

In [ ]:
nltk.download('punkt_tab')

In [ ]:
from nltk.tokenize import sent_tokenize

sentences = sent_tokenize(text)

sentences[0:5]

In [ ]:
nltk.download('averaged_perceptron_tagger_eng')

In [ ]:
import nltk

nltk.pos_tag(sentences)

In [ ]:
sentence = sentences[0]

sentence

In [ ]:
from nltk.tokenize import casual_tokenize

tokens = casual_tokenize(sentence)

tokens

In [ ]:
nltk.pos_tag(tokens)

In [ ]:
entities = []

for row in nltk.pos_tag(tokens):

    token = row[0]

    tag = row[1]

    if tag == 'NNP':

        entities.append(token)

entities

In [ ]:
def extract_entities(sentence):

    entities = []

    tokens = casual_tokenize(sentence)

    for row in nltk.pos_tag(tokens):

        token = row[0]

        tag = row[1]

        if tag == 'NNP':

            entities.append(token)

    return entities

In [ ]:
extract_entities(sentence)

In [ ]:
entities = [extract_entities(sentence) for sentence in sentences]

entities

In [ ]:
def extract_entities(sentence):

    entities = []

    tokens = casual_tokenize(sentence)

    for row in nltk.pos_tag(tokens):

        token = row[0]

        tag = row[1]

        if tag == 'NNP':

            entities.append(token)

    if len(entities) > 0:

        return entities

    else:

        return None

In [ ]:
entities = [extract_entities(sentence) for sentence in sentences]

entities[0:10]

In [ ]:
import pandas as pd

df = pd.DataFrame({'sentence':sentences, 'entities':entities})

df.head(10)

In [ ]:
df = df.dropna()

df = df[df['entities'].apply(len) > 1]

entities = df['entities'].to_list()

entities

In [ ]:
def extract_entities(sentence):

    entities = []

    tokens = casual_tokenize(sentence)

    for row in nltk.pos_tag(tokens):

        token = row[0]

        tag = row[1]

        if tag == 'NNP':

            if "'" in token:

                cutoff = token.index('\'')

                token = token[:cutoff]

            entities.append(token)

    if len(entities) > 0:

        return entities

    else:

        return None

In [ ]:
entities = [extract_entities(sentence) for sentence in sentences]

entities

In [ ]:
df = pd.DataFrame({'sentence':sentences, 'entities':entities})

df = df.dropna()

df = df[df['entities'].apply(len) > 1]

entities = df['entities'].to_list()

entities

In [ ]:
from string import punctuation

def extract_entities(sentence):

    entities = []

    tokens = casual_tokenize(sentence)

    for row in nltk.pos_tag(tokens):

        token = row[0]

        tag = row[1]

        if tag == 'NNP':

            for p in punctuation:

                if p in token:

                    cutoff = token.index(p)

                    token = token[:cutoff]

            if len(token) > 1:

                entities.append(token)

    if len(entities) > 0:

        return entities

    else:

        return None

In [ ]:
entities = [extract_entities(sentence) for sentence in sentences]

df = pd.DataFrame({'sentence':sentences, 'entities':entities})

df = df.dropna()

df = df[df['entities'].apply(len) > 1]

entities = df['entities'].to_list()

entities

In [ ]:
def get_book_entities():

    text = get_data()

    sentences = sent_tokenize(text)

    entities = [extract_entities(sentence) for sentence in sentences]

    df = pd.DataFrame({'sentence':sentences, 'entities':entities})

    df = df.dropna()

    df = df[df['entities'].apply(len) > 1]

    entities = df['entities'].to_list()

    return entities,df

In [ ]:
entities, df = get_book_entities()

entities[0:5]

In [ ]:
df

# spaCy Pos-Tagging

In [ ]:
!python -m spacy download en_core_web_md

In [ ]:
import spacy

nlp = spacy.load("en_core_web_md")

In [ ]:
def get_data():

    url = 'https://www.gutenberg.org/files/5200/5200-0.txt'
    text = requests.get(url).text

    # strip header junk
    cutoff = text.index('One morning')
    text = text[cutoff:]

    # strip footer junk
    cutoff = text.rindex('*** END OF THE PROJECT GUTENBERG EBOOK METAMORPHOSIS ***')
    text = text[:cutoff]

    # pre-processing to clean the text
    text = text.replace('\r', ' ').replace('\n', ' ')
    text = text.replace('â\x80\x99', '\'').replace('â\x80\x9c', '"').replace('â\x80\x9d', '""').replace('â\x80\x94', ' ')

    return text

In [ ]:
text = get_data()

text[0:279]

In [ ]:
doc = nlp(text)

sentences = list(doc.sents)

In [ ]:
for s in sentences[0:6]:

    print(s)

    print()

In [ ]:
for token in sentences[0]:

    print('{}: {}'.format(token.text, token.tag_))

In [ ]:
for token in sentences[0]:

    print('{}: {}'.format(token.text, token.pos_))

In [ ]:
entities = []

for token in sentences[0]:

    if token.tag_ == 'NNP':

        entities.append(token.text)

entities

In [ ]:
entities = []

for sentence in sentences:

    sentence_entities = []

    for token in sentence:

        if token.tag_ == 'NNP':

            sentence_entities.append(token.text)

    entities.append(sentence_entities)

entities[0:10]

In [ ]:
def extract_entities(text):

    doc = nlp(text)

    sentences = list(doc.sents)

    entities = []

    for sentence in sentences:

        sentence_entities = []

        for token in sentence:

            if token.tag_ == 'NNP':

                sentence_entities.append(token.text)

        if len(sentence_entities) > 0:

            entities.append(sentence_entities)

    return entities

In [ ]:
extract_entities(text)

# spaCy NER

In [ ]:
for token in sentences[0]:

    print('{}: {}'.format(token.text, token.ent_type_))

In [ ]:
sentences[0].text

In [ ]:
doc = nlp(sentences[0].text)

for ent in doc.ents:

    print('{}: {}'.format(ent, ent.label_))

In [ ]:
def extract_entities(text):

    doc = nlp(text)

    sentences = list(doc.sents)

    entities = []

    for sentence in sentences:

        sentence_entities = []

        sent_doc = nlp(sentence.text)

        for ent in sent_doc.ents:

            if ent.label_ in ['PERSON', 'ORG', 'GPE']:

                entity = ent.text.strip()

                if "'s" in entity:

                    cutoff = entity.index("'s")

                    entity = entity[:cutoff]

                if entity != '':

                    sentence_entities.append(entity)

        sentence_entities = list(set(sentence_entities))

        if len(sentence_entities) > 1:

            entities.append(sentence_entities)

    return entities

In [ ]:
text

In [ ]:
len(text)

In [ ]:
morph_entities = extract_entities(text)

morph_entities[10:20]

In [ ]:
len(morph_entities)

In [ ]:
morph_entities

In [ ]:
def get_data():

    url = 'https://www.gutenberg.org/files/11/11-0.txt'
    text = requests.get(url).text

    # strip header junk
    cutoff = text.index('Alice was beginning')
    text = text[cutoff:]

    # strip footer junk
    cutoff = text.rindex('THE END')
    text = text[:cutoff]

    # pre-processing to clean the text
    text = text.replace('\r', ' ').replace('\n', ' ')
    text = text.replace('â\x80\x99', '\'').replace('â\x80\x9c', '"').replace('â\x80\x9d', '""').replace('â\x80\x94', ' ')

    return text

In [ ]:
text = get_data()

text[0:310]

In [ ]:
alice_entities = extract_entities(text)

alice_entities[0:10]

# Converting entities to network data

In [ ]:
alice_entities[0:10]

In [ ]:
final_sources = []
final_targets = []

for row in alice_entities:

    source = row[0]
    targets = row[1:]

    for target in targets:

        final_sources.append(source)
        final_targets.append(target)

In [ ]:
final_sources[0:5]

In [ ]:
final_targets[0:5]

In [ ]:
def get_network_data(entities):

    final_sources = []
    final_targets = []

    for row in entities:

        source = row[0]
        targets = row[1:]

        for target in targets:

            final_sources.append(source)
            final_targets.append(target)

    df = pd.DataFrame({'source':final_sources, 'target':final_targets})

    return df

In [ ]:
alice_network_df = get_network_data(alice_entities)

alice_network_df.head()

In [ ]:
morph_network_df = get_network_data(morph_entities)

morph_network_df.head()

# Converting network data into networks

In [ ]:
import networkx as nx

G_alice = nx.from_pandas_edgelist(alice_network_df)

G_morph = nx.from_pandas_edgelist(morph_network_df)

In [ ]:
print(G_alice)

In [ ]:
print(G_morph)

In [ ]:
!pip install scikit-network

In [ ]:
nx.to_scipy_sparse_array

In [ ]:
def draw_graph(G, show_names=False, node_size=1, font_size=10, edge_width=0.5):

    import numpy as np
    import networkx as nx

    from IPython.display import SVG
    from sknetwork.visualization import svg_graph
    from sknetwork.data import Bunch
    from sknetwork.ranking import PageRank
    import scipy.sparse as sp

    adjacency = nx.adjacency_matrix(G, weight='weight')
    adjacency = sp.csr_matrix(adjacency)

    names = np.array(list(G.nodes()))

    graph = Bunch()
    graph.adjacency = adjacency
    graph.names = np.array(names)

    pagerank = PageRank()

    pagerank.fit(adjacency)
    scores = pagerank.scores_

    if show_names:

        image = svg_graph(graph.adjacency, font_size=font_size, node_size=node_size, names=graph.names, width=700, height=500, scores=scores, edge_width=edge_width)

    else:

        image = svg_graph(graph.adjacency, node_size=node_size, width=700, height=500, scores = scores, edge_width=edge_width)

    return SVG(image)

In [ ]:
def draw_ego_graph(G, ego, center=True, k=0, show_names=True, edge_width=0.1, node_size=3, font_size=12):

    ego = nx.ego_graph(G, ego, center=center)

    ego = nx.k_core(ego, k)

    return draw_graph(ego, node_size=node_size, font_size=font_size, show_names=show_names, edge_width=edge_width)

In [ ]:
draw_graph(G_alice)

In [ ]:
draw_graph(G_alice, edge_width=0.2, node_size=3, show_names=True)

In [ ]:
draw_graph(G_alice, edge_width=0.2, node_size=2, show_names=True, font_size=12)

In [ ]:
draw_ego_graph(G_alice, 'Alice')

In [ ]:
draw_ego_graph(G_alice, 'Alice', center=False)

In [ ]:
draw_ego_graph(G_alice, 'Alice', center=False, k=1)

In [ ]:
draw_graph(G_morph, show_names=True, node_size=3, font_size=12)

In [ ]:
morph_network_df

In [ ]:
morph_network_df.drop_duplicates()